In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
#내가 별도로 설치한 케라스를 사용하겠다.
import keras

Using TensorFlow backend.


In [2]:
#XOR문제를 해결하기 위한 데이터
#X에는 4*2행렬, 항상 행의 수가 샘플수고, 열의 수가 dimension
#Y도 행렬이다.
#행렬연산의 결과는 항상 행렬로 나온다.
#Y의 구조 행의 수 == 샘플수, 열의 수 == 클래스 수
#텐서플로우내부에 들어있는 케라스와 케라스 자체적으로 사용하는것 두가지가 있다. 둘다 다르다.


X = np.array([[0,0], [0,1], [1,0], [1,1]], 'float32')
Y = np.array([[0], [1], [1], [0]], 'float32')

#무슨 말이냐? 이렇게 쓰면 오류난다. 즉, 통일을 시켜주어야 한다.
#model = keras.models.Sequential()
#model.add(tf.keras.layers.Dense(64, input_dim=2, activation='relu'))
#model.add(tf.keras.layers.Dense(64, activation='relu'))

#이렇게 하면 된다.
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, input_dim=2, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))

#아님 텐서플로우에 내장된것만 사용해도 된다. 통일만 시켜주면 된다.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=2, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))

#맨 마지막은 0~1사이 값이 나오도록, sigmoid 사용
#그러면, Y = np.array([[0], [2], [2], [0]])으로 써도 상관없다.
#
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))

#sigmoid 레이어의 이름이 output으로 바뀐것을 볼 수 있다.
model.add(tf.keras.layers.Dense(1, activation ='sigmoid', name = 'output'))


model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])

model.fit(X, Y, batch_size=1, epochs=100, verbose=0)


model.summary()

#생성할때마다 이름이 바뀐다. 다시 실행시 그 다음부터
#내가 다른 사람들에게 배포할때는, 이름을 고정시켜 주어야 한다.

# inputs:  ['dense_input']
print('inputs: ', [input.op.name for input in model.inputs])

# outputs:  ['dense_4/Sigmoid']
print('outputs: ', [output.op.name for output in model.outputs])




model.save('xor.h5')

#텐서플로우를 백그라운드로, 케라스는 얹혀진 형태
#우리가 케라스로 보고 있는 이름과 실제, 텐서플로우에서 사용하는 이름은 다르다.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 12,737
Trainable params: 12,737
Non-trainable params: 0
_________________________________________________________________
inputs:  ['dense_input']
outputs:  ['

In [3]:
#꺽쇠가 두번이면 행렬, 11일때, 0, 01일때, 1이 나오는것을 확인할 수 있다.
model.predict(np.array([[1, 1], [0,1]]))

array([[0.00824266],
       [0.99328977]], dtype=float32)

In [4]:
from tensorflow.compat.v1.keras import backend as K
#텐서플로우를 버전1.0을 사용하겠다.
import tensorflow.compat.v1 as tf
#여기서부터 텐서플로우는 버전1.0이다. 이때, tf는 tensorflow.compat.v1이다.

#tf2버전은 사용하지 않겠다.
tf.disable_v2_behavior()

#여기는 자세히는 이해 안해도 됨
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph


#output node만 알고 있으면 된다.
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
#True면, 사람이 읽을 수 있는 텍스트
#False면 배열로 표시
tf.train.write_graph(frozen_graph, './', 'xor.pbtxt', as_text=True)
tf.train.write_graph(frozen_graph, './', 'xor.pb', as_text=False)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 39 variables.
INFO:tensorflow:Converted 39 variables to const ops.


'./xor.pb'

In [5]:
import cv2 as cv2
#텐서플로우로부터 가져온 모델을 사용하기 위해 쓰는것
net = cv2.dnn.readNetFromTensorflow('xor.pb')
#Layer들의 이름을 받아낸것을 변수에 저장
#이름/연산 이런 꼴로 나타난다.
layersNames = net.getLayerNames()
print(layersNames)
#실제 레이어는 보기보다 많다.

['dense/MatMul', 'dense/Relu', 'dense_1_1/MatMul', 'dense_1_1/Relu', 'dense_2_1/MatMul', 'dense_2_1/Relu', 'dense_3/MatMul', 'dense_3/Relu', 'output/MatMul', 'output/Sigmoid']


In [6]:
net.setInput(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]))

#마지막 출력Layer를 보려면, 마지막 행 이름으로 바꾸어 주어야한다.
out = net.forward(outputName='output/Sigmoid')
print(out)

#마지막까지 연산을 해주세요. 거꾸로 가는것은 backward
out = net.forward()
#print(out)

[[0.01501559]
 [0.99328977]
 [0.9908274 ]
 [0.00824266]]


# mnist 로딩
- 내가 keras를 pb로 변환해서 동작하는지 확인 필요

In [7]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 42s 701us/step - loss: 0.2643 - accuracy: 0.9181 - val_loss: 0.0536 - val_accuracy: 0.9828


In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Test loss: 0.05364830540092662
Test accuracy: 0.9828000068664551


In [9]:
model.save('mnist.h5')

In [10]:
# 일단 반드시 이와같이 해야함
import tensorflow as tf 
from tensorflow import keras
model3 = tf.keras.models.load_model("mnist.h5", compile=False)
model3.save("out", save_format='tf') # 폴더명임

INFO:tensorflow:Assets written to: out\assets


In [11]:
#model.save("mnist2.pb", save_format='tf')


from tensorflow import keras
model3 = keras.models.load_model("mnist.h5", compile=False)

export_path = 'saved_model.pb가 저장될 디렉토리'
model3.save("", save_format='tf')

#load_model 을 할 때 두 번째 인자에 compile 옵션을 안 넣으면, 내 모델이 컴파일이 안 되어 있다고 오류가 뜬다. 구글링해보니 컴파일이란 거는 학습할 준비 어쩌고 하던데 하여튼 학습을 하지 않고 추론하는 데에만 모델을 사용할 거라면, compile=False 옵션을 넣어주면 된다고 해서 넣었음.

#​

#그리고 export_path는 파일이 저장될 디렉토리를 지정해주어야 하는데, 디렉토리 안이 비어있어야 한다.

#그리하여 이것을 실행하면 !!
#[출처] Keras 모델을 TensorFlow Lite로 변환하기 (h5 -> pb -> tflite)|작성자 aainy


INFO:tensorflow:Assets written to: assets


In [12]:
from tensorflow.compat.v1.keras import backend as K

frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
#tf.train.write_graph(frozen_graph, './', 'xor.pbtxt', as_text=True)
tf.train.write_graph(frozen_graph, './', 'mnist.pb', as_text=False)

INFO:tensorflow:Froze 84 variables.
INFO:tensorflow:Converted 84 variables to const ops.


'./mnist.pb'

In [13]:
import tensorflow.compat.v1 as tf
from tensorflow.core import framework

def find_all_nodes(graph_def, **kwargs):
    for node in graph_def.node:
        for key, value in kwargs.items():
            if getattr(node, key) != value:
                break
        else:
            yield node
    raise StopIteration


def find_node(graph_def, **kwargs):
    try:
        return next(find_all_nodes(graph_def, **kwargs))
    except StopIteration:
        raise ValueError(
            'no node with attributes: {}'.format(
                ', '.join("'{}': {}".format(k, v) for k, v in kwargs.items())))


def walk_node_ancestors(graph_def, node_def, exclude=set()):
    openlist = list(node_def.input)
    closelist = set()
    while openlist:
        name = openlist.pop()
        if name not in exclude:
            node = find_node(graph_def, name=name)
            openlist += list(node.input)
            closelist.add(name)
    return closelist


def remove_nodes_by_name(graph_def, node_names):
    for i in reversed(range(len(graph_def.node))):
        if graph_def.node[i].name in node_names:
            del graph_def.node[i]


def make_shape_node_const(node_def, tensor_values):
    node_def.op = 'Const'
    node_def.ClearField('input')
    node_def.attr.clear()
    node_def.attr['dtype'].type = framework.types_pb2.DT_INT32
    tensor = node_def.attr['value'].tensor
    tensor.dtype = framework.types_pb2.DT_INT32
    tensor.tensor_shape.dim.add()
    tensor.tensor_shape.dim[0].size = len(tensor_values)
    for value in tensor_values:
        tensor.tensor_content += value.to_bytes(4, 'little')
    output_shape = node_def.attr['_output_shapes']
    output_shape.list.shape.add()
    output_shape.list.shape[0].dim.add()
    output_shape.list.shape[0].dim[0].size = len(tensor_values)


def make_cv2_compatible(graph_def):
    # A reshape node needs a shape node as its second input to know how it
    # should reshape its input tensor.
    # When exporting a model using Keras, this shape node is computed
    # dynamically using `Shape`, `StridedSlice` and `Pack` operators.
    # Unfortunately those operators are not supported yet by the OpenCV API.
    # The goal here is to remove all those unsupported nodes and hard-code the
    # shape layer as a const tensor instead.
    for reshape_node in find_all_nodes(graph_def, op='Reshape'):

        # Get a reference to the shape node
        shape_node = find_node(graph_def, name=reshape_node.input[1])

        # Find and remove all unsupported nodes
        garbage_nodes = walk_node_ancestors(graph_def, shape_node,
                                            exclude=[reshape_node.input[0]])
        remove_nodes_by_name(graph_def, garbage_nodes)

        # Infer the shape tensor from the reshape output tensor shape
        if not '_output_shapes' in reshape_node.attr:
            raise AttributeError(
                'cannot infer the shape node value from the reshape node. '
                'Please set the `add_shapes` argument to `True` when calling '
                'the `Session.graph.as_graph_def` method.')
        output_shape = reshape_node.attr['_output_shapes'].list.shape[0]
        output_shape = [dim.size for dim in output_shape.dim]

        # Hard-code the inferred shape in the shape node
        make_shape_node_const(shape_node, output_shape[1:])

In [15]:
sess = K.get_session()
graph_def = sess.graph.as_graph_def(add_shapes=True)
graph_def = tf.graph_util.convert_variables_to_constants(sess, graph_def, [model.output.name.split(':')[0]])
make_cv2_compatible(graph_def)

# Print the graph nodes
print('\n'.join(node.name for node in graph_def.node))

# Save the graph as a binary protobuf2 file
tf.train.write_graph(graph_def, '', 'mnist.pb', as_text=False)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


OverflowError: can't convert negative int to unsigned

In [16]:
#net = cv2.dnn.readNetFromTensorflow('MINIST_CNN_frozen_graph.pb')
import cv2 as cv2
net = cv2.dnn.readNetFromTensorflow('saved_model.pb')
layersNames = net.getLayerNames()
print(layersNames)

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_io.cpp:42: error: (-2:Unspecified error) FAILED: ReadProtoFromBinaryFile(param_file, param). Failed to parse GraphDef file: saved_model.pb in function 'cv::dnn::ReadTFNetParamsFromBinaryFileOrDie'


In [17]:
img = np.zeros((28,28), dtype=np.uint8)  #  이미지 일때는 uint8이어야 한다. 그래야 에러 없다.
blob = cv2.dnn.blobFromImage(img) #  학습할때 256으로 여기서 나누어야 한다.
print(blob.shape)
print(blob.dtype)

(1, 1, 28, 28)
float32


In [18]:
#N: number of images in the batch
#H: height of the image
#W: width of the image
#C: number of channels of the image
    
#blob = blob.reshape(1, 28,28,1)  #NCHW
blob = blob.reshape(1, 1, 28,28)  #NHWC
net.setInput(blob)
out = net.forward()
print(out)

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\dnn\src\layers\fully_connected_layer.cpp:154: error: (-215:Assertion failed) srcMat.dims == 2 && srcMat.cols == weights.cols && dstMat.rows == srcMat.rows && dstMat.cols == weights.rows && srcMat.type() == weights.type() && weights.type() == dstMat.type() && srcMat.type() == CV_32F && (biasMat.empty() || (biasMat.type() == srcMat.type() && biasMat.isContinuous() && (int)biasMat.total() == dstMat.cols)) in function 'cv::dnn::FullyConnectedLayerImpl::FullyConnected::run'


# CIFAR

In [19]:
from keras.models import load_model
model = load_model('cifar10.h5')
model.summary()

OSError: Unable to open file (unable to open file: name = 'cifar10.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [20]:
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, './', 'cifar.pb', as_text=False)
tf.train.write_graph(frozen_graph, './', 'cifar.pbtxt', as_text=True)

INFO:tensorflow:Froze 84 variables.
INFO:tensorflow:Converted 84 variables to const ops.


'./cifar.pbtxt'

In [21]:
net = cv2.dnn.readNetFromTensorflow('cifar.pb')
layersNames = net.getLayerNames()
print(layersNames)

['conv2d_1/convolution', 'conv2d_1/Relu', 'conv2d_2/convolution', 'conv2d_2/Relu', 'max_pooling2d_1/MaxPool', 'flatten_1/Shape', 'flatten_1/strided_slice', 'flatten_1/Prod', 'flatten_1/stack_360', 'flatten_1/Reshape', 'dense_3_1/MatMul', 'dense_3_1/Relu', 'dense_4/MatMul', 'dense_4/Softmax']


In [ ]:
#from keras.utils import to_categorical
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
print(test_images.shape)

In [ ]:
img = test_images[0,:,:,:] / 255
print(img.shape)

plt.imshow(img)
print(img)

In [ ]:
o = model.predict(img.reshape(1,32,32,3))

np.argmax(o)



In [ ]:
img = (img*255).astype(dtype='uint8')

blob = cv2.dnn.blobFromImage(img, 1/255) #  학습할때 256으로 나누었음.
print(blob.shape)
print(blob.dtype)

#why ? 보통 1,32,32,3으로 넘겨야함

In [ ]:
#
net.setInput(blob)
out = net.forward()
print(out)

In [ ]:
def freeze_graph(model_dir, output_node_names):
  """Extract the sub graph defined by the output nodes and convert 
  all its variables into constant 
  Args:
      model_dir: the root folder containing the checkpoint state file
      output_node_names: a string, containing all the output node's names, 
                          comma separated
                        """
  if not tf.gfile.Exists(model_dir):
    raise AssertionError(
      "Export directory doesn't exists. Please specify an export "
      "directory: %s" % model_dir)

  if not output_node_names:
    print("You need to supply the name of a node to --output_node_names.")
    return -1

  # We retrieve our checkpoint fullpath
  checkpoint = tf.train.get_checkpoint_state(model_dir)
  input_checkpoint = checkpoint.model_checkpoint_path
    
  # We precise the file fullname of our freezed graph
  absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
  output_graph = absolute_model_dir + "/frozen_model.pb"
  print(output_graph)

  # We clear devices to allow TensorFlow to control on which device it will load operations
  clear_devices = True

  # We start a session using a temporary fresh Graph
  with tf.Session(graph=tf.Graph()) as sess:
    # We import the meta graph in the current default Graph
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    # We restore the weights
    saver.restore(sess, input_checkpoint)

    # We use a built-in TF helper to export variables to constants
    output_graph_def = tf.graph_util.convert_variables_to_constants(
      sess, # The session is used to retrieve the weights
      tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
      output_node_names.split(",") # The output node names are used to select the usefull nodes
    ) 

    # Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile(output_graph, "wb") as f:
      f.write(output_graph_def.SerializeToString())
    print("%d ops in the final graph." % len(output_graph_def.node))

  return output_graph_def




In [ ]:
estimator_model = tf.keras.estimator.model_to_estimator(keras_model = model, model_dir = './k')
#k 폴더에 keras폴더에 chekc point 파일 저장함
# out이름 어덯게 알지?
freeze_graph('./k/keras/',"activation_3/Sigmoid")

#./k/keras/frozen_model.pb에 생성

""

In [ ]:
model.summary()

In [ ]:
model.layers[7].name